# Trends in E-Rate Requests from 2016 to 2018
## by: Tim Lupien

This report examines the year on year change in both the volume and the 
total value of E-Rate funding requests from 2016 to 2018. The
volume decreased by 20% and 25% while value decreased by 10% and 12%, for
2016-17 and 2017-18 resepectively. The overall trend can be overwhelmingly
attributed to a decrease in requests for voice service funding, an expected
consequence of the [E-Rate Modernization Order](https://www.fcc.gov/general/summary-e-rate-modernization-order)
which began phasing out discounts for voice services in 2014. The difference
in the rate of decrease between volume and value of requests is primarily
due to the outsized impact of the largest 1% of requesting organizations on
the total dollar value of requests. These organizations account for 6% of the
volume of requests, but 45% of the dollar value requested, and do not always
follow the same trends as their smaller counterparts.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sodapy import Socrata
import seaborn as sns

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# pretty colors
usac_light = '#d9ebff'
usac_dark = '#004bd3'
usac_grey = '#eeeeee'
banner_yellow = '#ffdd96'

In [3]:
# setup
client = Socrata("opendata.usac.org",\
                 "XibSDHkB4OvBkPQbvE8GKsK85")
data_id = "qdmp-ygft"

# query
where_clause = "form_version = 'Original'\
                AND is_certified_in_window = 'In Window'\
                AND funding_year between '2016' and '2018'"

# pages calculation
page_size = 1000
n_records = int(client.get(data_id,
                       select = 'COUNT(*)',
                       where = where_clause)[0]['COUNT'])
n_pages = n_records // page_size + 1
print(f'This query will result in {n_pages} of {page_size} results each.')

In [4]:
# page through results to gather all of them
# let me know if there's a better way of using the API...
# this is very slow, and I saved the resultant dataframe as
# a csv and would re-import it from that while working on
# this...
df_list = []
print('Starting download.')
for i in range(n_pages):
    page = client.get(data_id,
                      limit=page_size,
                      offset=i*page_size,
                      order=':id',
                      select = '*',
                      where = where_clause)
    page_df = pd.DataFrame.from_records(page)
    df_list.append(page_df)
    if i % 50 == 0:
        print(f'{i} pages ({i * page_size} observations) have been downloaded...')

# Concat on a list of dfs more efficient than repeated append calls
df = pd.concat(df_list)
print('Complete.')

In [5]:
def handle_long_labels(label_list):
    '''Takes a list of strings and returns the same list of strings but
    with line breaks inserted after the first however many words take
    up longer than 15 characters'''
    
    new_labels = []
    for label in label_list:
        if len(label) >= 30:
            label_words = label.split(' ')
            length = 0
            word_index = 0
            for word in label_words:
                length += len(word)
                word_index += 1
                if length >= 15:
                    break
            label_words[word_index] = '\n' + label_words[word_index]
            new_label = ' '.join(label_words)
        else:
            new_label = label
        new_labels.append(new_label)
    return new_labels

In [6]:
def nice_pareto(ax, d, cs, metric, cat_name, year):
    '''Makes pareto chart. Takes an axis (subplot), d (the absolute
    change), cs (the cumulative sum of percent change), and the metric
    and category name (for the dynamic title).'''
    
    # Branding
    usac_light = '#d9ebff'
    usac_dark = '#004bd3'
    
    # the bars
    ax.bar(d.index, -d, edgecolor = 'k', color=usac_light, fill=True)
    ax.xaxis.set_ticks(d.index)
    ax.set_xticklabels(handle_long_labels(d.index), rotation=-15)
    ax.set_ylabel('Absolute Size of Decrease')
    ax.set_xlabel('Category')
    ax.set_title(f'{year} Decrease in {metric} by {cat_name}')
    ax.axhline(0, c='k', lw=1)

    #the line
    rax = ax.twinx() #right axis
    rax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(
        xmax=1,
        decimals=0
    ))
    rax.set_ylabel('Cumulative Percent of Total Decrease')
    rax.set_ylim(0, cs.max() + 0.05)
    rax.plot(cs, c=usac_dark, marker='o', ms=8, lw=2)

In [7]:
def relative_change_plot(ax, pt, col, metric, cat_name, year):
    '''Makes a relative change plot (boxes around 0). pt is a pivot
    table (made in the master plot function) and col is a column of
    said pivot table corresponding to a certain year over year period's
    worth of relative change'''
    
    # Branding
    usac_light = '#d9ebff'
    usac_dark = '#004bd3'
    
    pt.sort_values(by=col, inplace=True) # sorting
    
    # actual plot
    ax.bar(pt.index,
           -pt[col],
           edgecolor=usac_dark,
           hatch='/',
           linewidth=1,
           fill=False)
    
    # ticks
    ax.xaxis.set_ticks(pt.index)
    ax.set_xticklabels(handle_long_labels(pt.index), rotation=-15)
    
    ax.yaxis.set_major_formatter(
        mpl.ticker.PercentFormatter(xmax=1, decimals=1))
    
    #labels
    ax.set_ylabel('Change Relative to Total Rate of Decrease\
    \n (positive is a faster decrease)')
    ax.set_xlabel('Category')
    ax.set_title(f'{year} Relative Change in {metric} by {cat_name}')
    ax.axhline(0, c='k', lw=1) # since they go up *and* down this is helpful

In [8]:
def master_plot_function(df, category, metric, cat_name):
    '''This function creates the necessary inputs to create the four
    plots I want for categorical variables (and makes the plots). It
    takes a (the) dataframe, a category (a column of categorical
    variables within the dataframe, a metric of either "Volume" or
    "Value", which is used for setup and to create titles, and a
    category name, which is a pretty version of the column name, also
    for titles.'''
    
    # setup stuff
    usac_light = '#d9ebff'
    usac_dark = '#004bd3'
    
    if metric == 'Volume':
        values = 'funding_request_number'
        agg = 'count'
        
    elif metric == 'Value':
        values = 'funding_commitment_request'
        agg = 'sum'  
        

        
    # important aggregations    
    total_change = df.groupby(by='funding_year')[values].agg(agg)       
        
    pt = df.pivot_table(values,
                    index=category,
                    columns='funding_year',
                    aggfunc=agg
                   )
    
    # absolute changes and cumulative for pareto
    
    # 2016-17 absolute change for bars
    d1617 = pt[2017] - pt[2016]
    d1617 = d1617.sort_values()
    
    # 2016-17 cumulative percent
    cs1617 = d1617.cumsum()
    total_change1617 = total_change[2017] - total_change[2016]
    cs1617 /= total_change1617
    
    # 2017-18 absolute change for bars
    d1718 = pt[2018] - pt[2017]
    d1718 = d1718.sort_values()
    
    # 2017-18 cumulative percent
    cs1718 = d1718.cumsum()
    total_change1718 = total_change[2018] - total_change[2017]
    cs1718 /= total_change1718
    
    # relative change
    
    # total percent change
    total_pct_chng_1617 = (total_change[2017]
                           - total_change[2016]) \
                           / total_change[2016]
    
    total_pct_chng_1718 = (total_change[2017]
                           - total_change[2016]) \
                           / total_change[2016]
    
    # absolute percent change for each category each period
    pt['pct_chng_1617'] = (pt[2017] - pt[2016]) / pt[2016]
    pt['pct_chng_1718'] = (pt[2018] - pt[2017]) / pt[2017]
    
    
    # the relative percent change
    pt['rel_pct_chng_1617'] = (pt['pct_chng_1617']
                               - total_pct_chng_1617)

    pt['rel_pct_chng_1718'] = (pt['pct_chng_1718'] 
                               -  total_pct_chng_1718)
    
    
    # plotting
    
    # grid set-up
    fig = plt.figure(figsize=(19, 14))
    grid = plt.GridSpec(14,19)
    
    # the pareto chart for 2016-17
    ax1 = fig.add_subplot(grid[:6, 10:])
    nice_pareto(ax1, d1617, cs1617, metric, cat_name, '2016-17')
    
    # the pareto chart for 2017-18
    ax2 = fig.add_subplot(grid[8:, 10:])
    nice_pareto(ax2, d1718, cs1718, metric, cat_name, '2017-18')
    
    # relative change plot 16-17
    ax3 = fig.add_subplot(grid[:6, :9])
    relative_change_plot(ax3,
                         pt,
                         'rel_pct_chng_1617',
                         metric,
                         cat_name,
                         '2016-17')
    
    #relative change plot 17-18
    ax4 = fig.add_subplot(grid[8:, :9])
    relative_change_plot(ax4,
                         pt,
                         'rel_pct_chng_1718',
                         metric, cat_name,
                         '2017-18')
    ;

## Changes in Volume of Requests

Below is shown the total volume, or count, of funding requests for the three
years in question. In order to determine underlying trends that may be driving
this trend of decreasing volume, categorical variables were the most useful.
(See [Appendix 1](#Outlier-Organizations) for further details as to why
continuous variables were not considered.) Two methods of quantifying how
important a specific category of a categorical variable are used. In the first
the rate of change in requests of a certain category is compared to the total
rate of change in requests to yield a relative rate of decrease. In the second,
the absolute decrease in each category is considered, with the goal of
identifying which categories explain the greatest amount of the change.

In [9]:
df['funding_year'] = df['funding_year'].astype('int64')
df['funding_commitment_request'] = df['funding_commitment_request'].astype('float64')

NameError: name 'df' is not defined

In [ ]:
n_requests = df.groupby(by='funding_year')['funding_request_number'].count()
n_pct_chng_1617 = (n_requests[2017] - n_requests[2016]) / n_requests[2016]
n_pct_chng_1718 = (n_requests[2018] - n_requests[2017]) / n_requests[2017]

fig = plt.figure(figsize=(9,6))
ax = plt.axes()
bar1 = ax.bar(n_requests.index, n_requests, color=usac_dark)
ax.annotate(f'{round(n_pct_chng_1617 * 100, 2)}%',
            (2016.5, (bar1.patches[0].get_height() 
                      + bar1.patches[1].get_height())/2 ))
ax.annotate(f'{round(n_pct_chng_1718 * 100, 2)}%',
            (2017.5, (bar1.patches[1].get_height() 
                      + bar1.patches[2].get_height())/2 ))
ax.set_xticks([2016,2017,2018])
ax.set_title('Yearly Volume of Funding Requests')
ax.set_ylabel('Volume of Funding Requests');

### Service Type
Service Type proves to be the most relevant variable to the trend. As shown in
the charts below, the volume of requests for the voice service type decreased
at a faster rate than the total volume of requests. Indeed, a *much* faster
rate of more than 47% for 2017-18. The absolute decrease in voice service
requests also made up the vast majority (over 65% and over 90%, respectively)
of the total decrease for both years.

The combination of both a higher relative rate of change and a significant
absolute change make this the most relevant trend to understanding the overall
trend in volume.

In [ ]:
master_plot_function(df,
                     'form_471_service_type_name',
                     'Volume',
                     'Service Type')

### Organization Type
The changes in organization type say more about the underlying distribution of
requests by organization type than they do about any driving trend. School
Districts alone account for as huge a portion of the decrease as voice service
requests did, but the relative percent change in school districts was a mere
2% in 2016-17 and 6% in 2017-18. This suggests that, *quite unlike* with voice
service, the change in school district requests is not indicative of some
underlying effect.

Another interesting thing to note with this category is the consistently
slower decrease of Consortium-based requests. Unfortunately, Consortiums do
not make up a significant enough of the actual volume to be in any way
explanatory of the general trend.

In [ ]:
master_plot_function(df,
                     'organization_entity_type_name',
                     'Volume',
                     'Service Type')

### Contract Type

The trend with contract type is one "away" from tariffs and month-to-month
and "towards" contracts. While all are still decreasing with the general
trend, the consistant slower decrease in contract as opposed to the much
faster decrease in the others indicated that some change is occuring that
makes the former two kinds less favorable than contracts. This effect is
shown to be at least somewhat related to the trend in decreasing voice
service requests as the correlation between voice service and a
contract is -0.46, a moderate inverse relationship. In short, voice services
are at least somewhat less likely to have a contract (and thus, presumably
somewhat more likely to be month-to-month or tariff), and thus this trend
only adds credence to the idea that the overall trend is primarily caused
by the E-Rate Modernization.

In [ ]:
master_plot_function(df,
                     'contract_type_name',
                     'Volume',
                     'Service Type')

## Change in Value of Requests
In the most general possible sense, the change in the total value of requests
followed the same trend as the change in volume. This makes sense intuitively,
as, all else equal, fewer requests would mean fewer dollars requested. If this
logic holds, the change in value could be explained in much the same way as
the change in volume, as it would merely be a consequence thereof. However,
the interesting part here is that the dollar value requested actually
decreased at a slower rate.

In [ ]:
request_value = df.groupby(by='funding_year')['funding_commitment_request'].sum()

val_pct_chng_1617 = (request_value[2017] - request_value[2016])
                    / request_value[2016]
    
val_pct_chng_1718 = (request_value[2018] - request_value[2017])
                    / request_value[2017]

fig = plt.figure(figsize=(9,6))
ax = plt.axes()
bar2 = ax.bar(request_value.index, request_value / 10**9, color=usac_dark)
ax.annotate(f'{round(val_pct_chng_1617 * 100, 2)}%',
            (2016.5, (bar2.patches[0].get_height() 
                      + bar2.patches[1].get_height())/2 ))
ax.annotate(f'{round(val_pct_chng_1718 * 100, 2)}%',
            (2017.5, (bar2.patches[1].get_height() 
                      + bar2.patches[2].get_height())/2 ))

ax.set_xticks([2016,2017,2018])
ax.set_title('Yearly Dollar Value Requested')
ax.set_ylabel('Billions USD');

### Accounting for Organization Size
The primary reason this rate of change is so different from that of the volume
of requests is the outsized effect of extremely large organizations and their
individual decisions on the total value requested. Below is a kernel density
estimate illustrating the extreme skew of the distribution of organizations
by the amount of funding they requested over the entire period.

In [ ]:
orgs_val = df.groupby(by='organization_name') \
['funding_commitment_request'].sum()

fig = plt.figure(figsize=(9,6))
ax = plt.axes()
sns.kdeplot(orgs_val / 10**6, ax=ax, fill=True, color=usac_dark)
ax.set_title('Distribution of Funding Request Value')
ax.set_xlabel('Funding Requested (Millions USD)')
ax.set_ylabel('Density');

In [ ]:
orgs_val_yr = df.pivot_table('funding_commitment_request',
                             columns='funding_year',
                             index='organization_name',
                             aggfunc='sum',
                             margins=True) \
.sort_values(by='All',ascending=False)
orgs_val_yr.fillna(0, inplace=True)
orgs_val_yr.drop('All', axis = 0, inplace=True)

In [ ]:
# categories for organization size
df['organization_size'] = 'Small'

df.loc[(
    df['organization_name'].isin(orgs_val_yr[(orgs_val_yr['All']
    >= orgs_val.quantile(0.9))].index)
), 'organization_size'] = 'Large'

df.loc[(
    df['organization_name'].isin(orgs_val_yr[(orgs_val_yr['All']
    >= orgs_val.quantile(0.99))].index)),
    'organization_size'] = 'Very Large'

df.loc[(
    df['organization_name'].isin(orgs_val_yr[(orgs_val_yr['All']
    >= orgs_val.quantile(0.999))].index)),
    'organization_size'] = 'Outlier'

In light of this, it was necessary to categorize organizations by size. The
number of organizations in each category is presented in the table below. The
definition of each category is as follows:

- "Small" organizations are the bottom 90% of requestors.
- "Large" organizations are from 90-99%.
- "Very Large" organizations are from 99-99.9%
- "Outlier" organizations are the top 0.1% of requestors.

In [ ]:
df['organization_size'].value_counts()

The donut charts below illustrate just how massive the impact of the
largest organizations is on the total value requested. Together, the
top 10% of organizations made 28% of the requests by volume, but accounted
for a whopping 76% of the requests by value.

In [ ]:
large_requests = 
    df.groupby(by='organization_size')['funding_commitment_request'].sum()
    
large_requestors = 
    df.groupby(by='organization_size')['funding_request_number'].count()

In [ ]:
def nice_donut(ax, pt, title, mvpct='no'):
    '''Makes a donut chart from a binary category for the express purpose
    of not having the same code typed out twice'''
    wedges, texts, autopct = ax.pie(pt,
                       wedgeprops = {'width':0.25, 'edgecolor':'black'},
                       colors=[usac_dark, banner_yellow, usac_light, usac_grey],
                       startangle=0,
                       autopct='%1.1f%%'
                      )
    
    if mvpct == 'yes':
        wedge = wedges[1]
        txt = autopct[1]
        # the angle at which the text is located
        ang = (wedge.theta2 + wedge.theta1) / 2.
        x = wedge.r * 1.1 * np.cos(ang*np.pi/180)
        y = wedge.r * 1.1 * np.sin(ang*np.pi/180)
        txt.set_position((x, y))
        
        wedge = wedges[3]
        txt = autopct[3]
        # the angle at which the text is located
        ang = (wedge.theta2 + wedge.theta1) / 2.
        x = wedge.r * 1.1 * np.cos(ang*np.pi/180)
        y = wedge.r * 1.1 * np.sin(ang*np.pi/180)
        txt.set_position((x, y))

    
    ax.legend(wedges, pt.index,
          title='Organization Size',
          loc='center'
         )

    ax.set_title(title);

In [ ]:
fig = plt.figure(figsize=(12,6))
grid = plt.GridSpec(5,10, wspace=0.5)

ax1 = fig.add_subplot(grid[:,:5])
nice_donut(ax1, large_requestors,
           'Request Volume by Organization Size',
          mvpct='yes')

ax2 = fig.add_subplot(grid[:, 4:])
nice_donut(ax2, large_requests,
           'Request Value by Organization Size')

In order to gauge the degree to which each size category of organization
fits with the general trend of values decreasing, a view of the initial
chart depicting the overall yearly trend is reproduced below for each.
From this, it is clear that Outliers, which, again, account for 19% of the
total value, do not follow the overall trend at all. Very Large organizations
also show a departure from the general trend, by remaining almos the same for
2017-18.

In [ ]:
def value_bar(ax, df, request_size,):
    '''Recreate the bar chart for yearly value change but filtered to
    a specific size category'''
    request_value = df[(df['organization_size'] == request_size)] \
    .groupby(by='funding_year') \
    ['funding_commitment_request'] \
    .sum()
    
    val_pct_chng_1617 = (request_value[2017] - request_value[2016])
                        / request_value[2016]
        
    val_pct_chng_1718 = (request_value[2018] - request_value[2017]) 
                        / request_value[2017]

    bar2 = ax.bar(request_value.index, request_value / 10**9, color=usac_light)
    ax.annotate(f'{round(val_pct_chng_1617 * 100, 2)}%',
                (2016.5, (bar2.patches[0].get_height() 
                          + bar2.patches[1].get_height())/2 ))
    ax.annotate(f'{round(val_pct_chng_1718 * 100, 2)}%',
                (2017.5, (bar2.patches[1].get_height() 
                          + bar2.patches[2].get_height())/2 ))

    ax.set_xticks([2016,2017,2018])
    ax.set_title(f'Yearly Dollar Value Requested ({request_size})')
    ax.set_ylabel('Billions USD');

In [ ]:
fig = plt.figure(figsize=(13,10))
grid = plt.GridSpec(10,13)

ax1 = fig.add_subplot(grid[:5,:6])
value_bar(ax1, df, 'Small')

ax2 = fig.add_subplot(grid[:5, 7:])
value_bar(ax2, df, 'Large')

ax3 = fig.add_subplot(grid[6:, :6])
value_bar(ax3, df, 'Very Large')


ax4 = fig.add_subplot(grid[6:, 7:])
value_bar(ax4, df, 'Outlier')

All of this is evidence that the impact of extremely large organizations is the
primary reason behind the difference in rates of volume and value decrease. In light
of that, the next portions, which examine the same categorical variables as were
analyzed for volume, use only Small and Large organizations, or the bottom 99%.

In [ ]:
mask = (df['organization_size'] == 'Large') | (df['organization_size'] == 'Small')

### Service Type
The extremely well-supported trend caused by the voice service requests
being phased out intentionally holds true for value as well as volume.

What is surprising here, however, is Internal Connections. While this can
mostly be explained by Internal Connections being the highest value category
in general of service type, as evidenced by the lower relative rate of decrease
compared to voice, but higher absolute decrease, some exogenous shock must still
have occured in 2016-17 to account for the sheer scope of internal connections
requests decreasing. This would be an interesting topic for further analysis, but
was outside the breadth of this report.

In [ ]:
master_plot_function(df[mask],
                     'form_471_service_type_name',
                     'Value',
                     'Service Type')

### Organization Type
Unsurprisingly, for ogranization type school districts were again the largest
absolute change by far. This can be attributed both to the sheer volume of
school district requests, as shown in the previous section, and the greater
average funding requested by school districts, which tend to be larger entities 
than organizations of other types.

In [ ]:
master_plot_function(df[mask],
                     'organization_entity_type_name',
                     'Value',
                     'Organization Type')

### Contact Type
For contracts, the trend discovered when looking at volume also appears to
hold true overall, including the characteristic of being significantly less
pronounced in 2016-17 than in 2017-18.

In [ ]:
master_plot_function(df[mask],
                     'contract_type_name',
                     'Value',
                     'Contract Type')

## Conclusion

Overall, the main takeways from looking at the year on year changes in
the quantity and dollar value of E-Rate funding requests are:
- Voice requests are driving the decrease, and this is because of E-Rate Modernization
- The distribution of organization types does not change substantially
- Contract types are changing in accordance with the propensity for voice services to involve month-to-month or tariff contracts
- The top 1% of organizations and their non-systemic effects account for the difference in rates of decrease between volume and value.

## Apendicies

### Outlier Organizations

This table shows the "outlier" organizations by year. Note the tendency for such
organizations to have excessive costs in *only* a single year, before going back
down to almost zero (in some cases). This is indicative of massive one-time projects.

In [ ]:
round(orgs_val_yr[(orgs_val_yr['All'] >= orgs_val.quantile(0.999))] / 10**6, 2)

### Continuous Variables
The many continous (numeric) variables in the dataset also
exhibited some change over the years. Since most of them are
cost related, however, there is extreme skew. Categorical variables
made for a much more coherent narrative and comprehensible analysis.

The year on year changes in some summary statistics for the continuous
variables is reproduced below.

The increasing discount percent and corresponding increasing pre-discount
costs was interesting, but they roughly cancel each other out, so it was
not going to be explicative of the trend in value.

The one-time costs vs. recurring costs was also just potentially interesting
in general, especially once the trend of massive outlier one-time projects
alluded to above was discovered.

In [ ]:
cont = [
    "bid_count",
    "total_monthly_recurring_cost",
    "total_monthly_recurring_ineligible_costs",
    "total_monthly_recurring_eligible_costs",
    "months_of_service",
    "total_pre_discount_eligible_recurring_costs",
    "total_one_time_costs",
    "total_ineligible_one_time_costs",
    "total_pre_discount_eligible_one_time_costs",
    "total_pre_discount_costs",
    "dis_pct"
]

df[cont] = df[cont].astype('float64')

cont_groupby_dict = {}

for col in cont:
    gb = df.groupby(by='funding_year')[col].agg([
        'median',
        'mean',
        'skew',
    ])
    print(col)
    print(round(gb,2))
    print('\n\n')

This scatterplot illustrates the low overlap between one-time and
recurring costs. From it can be seen that there really is a phenomenon
of massive one-time projects (with outliers appearing to be 3-6x the size of
outliers for recurring costs).

In [ ]:
fig = plt.figure(figsize=(9,6))
ax = plt.axes()
ax.scatter(df['total_one_time_costs'] / 10**6,
           df['total_monthly_recurring_cost'] / 10**6,
           marker='o', c=usac_dark, s=5)
ax.set_title('Low Overlap between One-time and Recurring costs')
ax.set_ylabel('Total Monthly Recurring Costs \n\
(Millions USD)')
ax.set_xlabel('Total One-time Costs\n\
(Millions USD)');

### Voice and Contract Correlation

In [ ]:
df['is_voice'] = 0
df.loc[(df['form_471_service_type_name'] == 'Voice'), 'is_voice'] = 1

df['is_contract'] = 0
df.loc[(df['contract_type_name'] == 'Contract'), 'is_contract'] = 1

print(df['is_voice'].corr(df['is_contract']))